# Doğrusal Regresyon Egzersizleri

50 adet Startup'ın araştırma ve geliştirmeye yönelik harcaması, yönetime yönelik harcaması, pazarlama harcaması, kazandıkları kar miktarı ve kuruldukları lokasyon bilgisi bulunmaktadır. Amaç kar miktarını tahmin etmektir. Bu bir sayısal tahmin problemidir ve bağımlı değişkenimiz "Profit".

Numpy, matplotlib.pyplot, pandas ve seaborn kütüphanelerini çekirdeğe dahil edelim.

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings("ignore")

Dizinde bulunan veri çerçevemizi startups değişkenine atayalım. startups değişkenini df değişkenine kopyalayarak kullanmaya başlayalım.

In [ ]:
startups = pd.read_csv('../input/50-startupscsv/50_Startups.csv')
df = startups.copy()

İlk 5 gözlemini yazdıralım.

In [ ]:
df.head()

Son 5 gözlemini yazdıralım.

In [ ]:
df.tail()

Veri çerçevesinin bilgilerini görüntüleyelim.

In [ ]:
df.info()

Kaç gözlem ve öznitelikten oluştuğunu görüntüleyelim.

In [ ]:
df.shape

Eksik verileri kontrol edelim.

In [ ]:
df.isna().sum()

Korelasyon matrisi çizdirelim.

In [ ]:
df.corr()

Seaborn ile korelasyon matrisinin ısı haritasını çizdirelim.

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(df.corr(), annot=True, linewidths=1, ax=ax, xticklabels=True, yticklabels=True);

R&D Spend ve Profit arasındaki korelasyonu daha iyi görebilmek için scatterplot çizdirelim.

In [ ]:
sns.scatterplot(x="R&D Spend", y="Profit", data=df);

In [ ]:
sns.lmplot(x="R&D Spend", y="Profit", data=df)

Sayısal değişkenlerin dağılımını görmek için df üzerinden histogram çizdirelim.

In [ ]:
df.hist(figsize = (13,10))
plt.show()

Veri çerçevesinin temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

State'a ait benzersiz değerleri görüntüleyelim.

In [ ]:
df["State"].unique()

get_dummies yardımıyla State'a dair kategorik öznitelik çıkarımlarında bulunalım. Çünkü State'ların birbirine üstünlüğü yok, nominaller. Ordinal değil.

In [ ]:
pd.get_dummies(df.State)

In [ ]:
dummy = pd.get_dummies(df.State, prefix='State')

In [ ]:
df = pd.concat([df, dummy], axis=1)

In [ ]:
df.head()

State özniteliğini silip dummy olarak yaratılan State'lardan da birisini hariç tutarak veri çerçevemizi güncelleyelim.

In [ ]:
df = startups
df=pd.get_dummies(df, columns=['State'], drop_first=True)

In [ ]:
df.head()

Veri çerçevemizi bağımlı ve bağımsız değişkenler olmak üzere bölütleyelim.

In [ ]:
X = df.drop("Profit", axis = 1)
y = df["Profit"]

Bağımlı ve bağımsız değişkenleri kontrol edelim.

In [ ]:
X.head() # bağımsız değişkenlerimiz

In [ ]:
y.head() # bağımlı değişkenimiz Profit ( Kazandıkları kar miktarı)

Bu bağımlı ve bağımsız değişkenlerden train ve test olmak üzere 4 parça oluşturalım. Bunu yapmak için train_test_split kullanalım.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

4 parça değişkeni kontrol edelim.

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

LinearRegression'u çekirdeğe dahil edip modeli inşa edelim.

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

Modeli eğitmek için bağımlı bağımsız değişkenlerden oluşturulan eğitim verilerini verelim.

In [ ]:
model = lm.fit(X_train, y_train)

Modele daha önce görmediği bağımlı test değişkenini tahmin ettirelim. Bu tahmin değerlerimizi y_pred değişkenine atayalım.

In [ ]:
y_pred = model.predict(X_test)
y_pred

Tahminleri ve gerçek değerleri bir veri çerçevesinde toplayıp üzerinde göz gezdirelim.

In [ ]:
test_df = pd.DataFrame({'Gerçek Kar':y_test, 'Tahmin Edilen Kar':y_pred})
test_df

sklearn bünyesinde barınan metrics'i çekirdeğe dahil edelim ve MAE, MSE, RMSE değerlerini görüntüleyelim.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('Mean Squared Error (MSE) : ', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE) : ', np.sqrt(mean_squared_error(y_test, y_pred)))
print('Mean Absolute Error (MAE) : ', mean_absolute_error(y_test, y_pred))

Modelin R Squared değerini eğitim verileri üzerinden yazdıralım.

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train, model.predict(X_train))

In [ ]:
model.score(X,y)

Dileyenler statsmodel kullanarak hangi özniteliklerin model için %95 güvenilirlikle ne kadar anlamlı olup olmadığına da bakabilir. Modelde bazı feature selection işlemleri yaparak tekrardan eğitip yeni sonuçlar mukayese edilebilir.

In [ ]:
import statsmodels.api as sm
stmodel = sm.OLS(y, X).fit()
stmodel.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['Features'] = X_train.columns
vif['VIF'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif


In [ ]:
X = df.drop(['State_New York','State_Florida','Administration','Profit'], axis = 1)
y = df["Profit"]

In [ ]:
X.head() 

In [ ]:
y.head() 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [ ]:
model2 = lm.fit(X_train, y_train)

In [ ]:
y_pred = model2.predict(X_test)
y_pred

In [ ]:
test_df = pd.DataFrame({'Gerçek Kar':y_test, 'Tahmin Edilen Kar':y_pred})
test_df

In [ ]:
model2.score(X,y)